### Decorators in python essentially allow you to modify or enhance the behavior of a function or a class without making any changes to its code

In [ ]:
@decorator_name
def my_function():
    pass

### Python functions are first class objects, which means that they can be assigned to variables, passed in as arguments to other functions, returned from other functions, and stored in data structures

In [ ]:
def greet(name):
    return f"Hello, {name}!"

# Functions are objects - you can assign them to variables
my_function = greet
print(my_function("Alice"))  # Hello, Alice!

# You can store them in lists
function_list = [greet, print, len]

# You can pass them as arguments
def call_function(func, arg):
    return func(arg)

result = call_function(greet, "Bob")  # Hello, Bob!

### You can also define functions inside of other functions
### This is called a closure, where the inner function remembers the variables from the outer function;s scope

In [ ]:
def outer_function(message):
    def inner_function():
        print(f"Inner says: {message}")
    
    return inner_function

# Create a new function
my_func = outer_function("Hello from inner!")
my_func()  # Inner says: Hello from inner!

### Here’s an example of what a decorator would do, but not in the syntax of a decorator

In [ ]:
def my_function():
    print("Doing some work...")

# I want to add timing to this function
import time

def timing_wrapper(func):
    def wrapper():
        start_time = time.time()
        func()  # Call the original function
        end_time = time.time()
        print(f"Function took {end_time - start_time:.4f} seconds")
    return wrapper

# Manually "decorate" the function
my_function = timing_wrapper(my_function)
my_function()
# Output:
# Doing some work...
# Function took 0.0001 seconds

### Here’s what this would look like if you created it using the decorator syntax

In [1]:
import time

def timing_decorator(func):
    def wrapper():
        start_time = time.time()
        func()
        end_time = time.time()
        print(f"Function took {end_time - start_time:.4f} seconds")
    return wrapper

# This...
@timing_decorator
def my_function():
    print("Doing some work...")

# Is exactly equivalent to this:
# my_function = timing_decorator(my_function)

my_function()  # Automatically timed!

Doing some work...
Function took 0.0001 seconds


## Basic Decorator Examples
### Simple Logging Calls

In [ ]:
def log_calls(func):
    def wrapper():
        print(f"Calling function: {func.__name__}")
        result = func()
        print(f"Finished calling: {func.__name__}")
        return result
    return wrapper

@log_calls
def say_hello():
    print("Hello!")
    return "greeting_complete"

result = say_hello()
# Output:
# Calling function: say_hello
# Hello!
# Finished calling: say_hello

### Validation Decorator

In [ ]:
def validate_positive(func):
    def wrapper(number):
        if number <= 0:
            raise ValueError("Number must be positive!")
        return func(number)
    return wrapper

@validate_positive
def square_root(n):
    return n ** 0.5

print(square_root(16))  # 4.0
# square_root(-4)  # Would raise ValueError

### Handling Decorator Arguments with *args and **kwargs

In [ ]:
def debug_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

@debug_decorator
def add(a, b):
    return a + b

@debug_decorator
def greet(name, greeting="Hello"):
    return f"{greeting}, {name}!"

print(add(5, 3))
# Output:
# Calling add with args: (5, 3), kwargs: {}
# add returned: 8
# 8

print(greet("Alice", greeting="Hi"))
# Output:
# Calling greet with args: ('Alice',), kwargs: {'greeting': 'Hi'}
# greet returned: Hi, Alice!
# Hi, Alice!

## Decorators with Parameters
### This is called a Decorator Factory, a function that returns a decorator

In [ ]:
def repeat(times):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(3)
def say_hello():
    print("Hello!")

say_hello()
# Output:
# Hello!
# Hello!
# Hello!

### More Complex Parameterized Decorators

In [4]:

def retry(max_attempts=3, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            import time
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_attempts - 1:
                        raise e
                    print(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay}s...")
                    time.sleep(delay)
        return wrapper
    return decorator

@retry(max_attempts=3, delay=0.5)
def unreliable_function():
    import random
    if random.random() < 0.7:  # 70% chance of failure
        raise Exception("Random failure!")
    return "Success!"

# This will retry up to 3 times with 0.5s delay between attempts

### Preserving function metadata -> Normally, decorators replace the original function, which causes it to lose its metadata

In [ ]:
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@simple_decorator
def example_function():
    """This is an example function."""
    pass

print(example_function.__name__)    # wrapper (not example_function!)
print(example_function.__doc__)     # None (lost the docstring!)

### functools.wraps can help you prevent this from happening, preserving metadata

In [ ]:
from functools import wraps

def better_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@better_decorator
def example_function():
    """This is an example function."""
    pass

print(example_function.__name__)    # example_function
print(example_function.__doc__)     # This is an example function.

## Decorator Examples
### Caching/Memoization

In [ ]:
from functools import wraps

def memoize(func):
    cache = {}
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Create a key from arguments
        key = str(args) + str(sorted(kwargs.items()))
        
        if key not in cache:
            cache[key] = func(*args, **kwargs)
            print(f"Computing {func.__name__}{args}")
        else:
            print(f"Using cached result for {func.__name__}{args}")
        
        return cache[key]
    return wrapper

@memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))  # Will show which calculations are cached

### Authentication/Authorization

In [ ]:
def requires_auth(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # In a real app, you'd check session, token, etc.
        current_user = get_current_user()  # Hypothetical function
        if not current_user:
            raise PermissionError("Authentication required")
        return func(*args, **kwargs)
    return wrapper

@requires_auth
def delete_user_data(user_id):
    # Only runs if user is authenticated
    print(f"Deleting data for user {user_id}")

### Rate Limiting

In [ ]:
import time
from functools import wraps

def rate_limit(calls_per_second=1):
    def decorator(func):
        last_called = [0.0]  # Use list to make it mutable
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            elapsed = time.time() - last_called[0]
            wait_time = 1.0 / calls_per_second - elapsed
            
            if wait_time > 0:
                time.sleep(wait_time)
            
            last_called[0] = time.time()
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(calls_per_second=2)
def api_call():
    print(f"API called at {time.time()}")

# Can only be called twice per second
for i in range(5):
    api_call()

### Performance Monitoring

In [ ]:
import time
import functools

def performance_monitor(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        start_memory = get_memory_usage()  # Hypothetical function
        
        try:
            result = func(*args, **kwargs)
            status = "SUCCESS"
        except Exception as e:
            result = None
            status = f"ERROR: {e}"
            raise
        finally:
            end_time = time.perf_counter()
            end_memory = get_memory_usage()
            
            print(f"Function: {func.__name__}")
            print(f"Status: {status}")
            print(f"Time: {(end_time - start_time)*1000:.2f}ms")
            print(f"Memory: {end_memory - start_memory}MB")
            
        return result
    return wrapper

@performance_monitor
def complex_calculation(n):
    return sum(i**2 for i in range(n))

### Class Decorators

In [ ]:
def add_string_method(cls):
    def __str__(self):
        return f"{cls.__name__} instance"
    
    cls.__str__ = __str__
    return cls

@add_string_method
class MyClass:
    def __init__(self, value):
        self.value = value

obj = MyClass(42)
print(obj)  # MyClass instance

### Singleton pattern with class decorator

In [ ]:
def singleton(cls):
    instances = {}
    
    @wraps(cls)
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class DatabaseConnection:
    def __init__(self):
        print("Creating database connection")
    
    def query(self, sql):
        return f"Executing: {sql}"

# Only one instance will ever be created
db1 = DatabaseConnection()  # Creating database connection
db2 = DatabaseConnection()  # (no output - reuses instance)
print(db1 is db2)  # True

### You can also stack multiple Decorators

In [ ]:
def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"<b>{result}</b>"
    return wrapper

def italic(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"<i>{result}</i>"
    return wrapper

@bold
@italic
def say_hello(name):
    return f"Hello, {name}!"

print(say_hello("Alice"))  # <b><i>Hello, Alice!</i></b>

### These decorators are applied from bottom to top (Inner to Outer)

In [ ]:
# This:
@bold
@italic
def say_hello(name):
    return f"Hello, {name}!"

# Is equivalent to:
say_hello = bold(italic(say_hello))

### You can also use classes as decorators

In [ ]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.count = 0
        functools.update_wrapper(self, func)
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call {self.count} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))  # Call 1 to greet
print(greet("Bob"))    # Call 2 to greet
print(greet.count)     # 2

### You can also create contextual decorators

In [ ]:
import threading

def synchronized(func):
    func._lock = threading.Lock()
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        with func._lock:
            return func(*args, **kwargs)
    return wrapper

@synchronized
def thread_safe_function():
    # This function can only be executed by one thread at a time
    pass

### Some decorators can also act like properties

In [ ]:
class lazy_property:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__
    
    def __get__(self, obj, cls):
        if obj is None:
            return self
        
        # Calculate the value once and store it
        value = self.func(obj)
        setattr(obj, self.name, value)
        return value

class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @lazy_property
    def area(self):
        print("Calculating area...")  # This will only print once
        return 3.14159 * self.radius ** 2

circle = Circle(5)
print(circle.area)  # Calculating area... 78.53975
print(circle.area)  # 78.53975 (no calculation message)

## Some Common Pitfalls for Decorators
### The Late Binding Problem

In [ ]:
# WRONG - all decorators will reference the same variable
decorators = []
for i in range(3):
    def make_decorator():
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                print(f"Decorator {i}")  # i will always be 2!
                return func(*args, **kwargs)
            return wrapper
        return decorator
    decorators.append(make_decorator())

# CORRECT - capture the current value
decorators = []
for i in range(3):
    def make_decorator(num=i):  # Capture i as default argument
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                print(f"Decorator {num}")
                return func(*args, **kwargs)
            return wrapper
        return decorator
    decorators.append(make_decorator())

### Always use functools.wraps

In [ ]:
from functools import wraps

def good_decorator(func):
    @wraps(func)  # Always include this!
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

### Handle edge cases

In [ ]:
def robust_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            # Your decorator logic here
            return func(*args, **kwargs)
        except Exception as e:
            # Decide how to handle exceptions
            # Re-raise, log, return default value, etc.
            raise
    return wrapper

### Keep decorators simple

In [ ]:
# DON'T - too much logic in the decorator
def overly_complex_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        # 50 lines of complex logic
        pass
    return wrapper

# DO - extract complex logic to separate functions
def helper_function():
    # Complex logic here
    pass

def clean_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        helper_function()
        return func(*args, **kwargs)
    return wrapper

## Python also has many built in decorators that you can use
### Property

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        if value < 0:
            raise ValueError("Radius cannot be negative")
        self._radius = value
    
    @property
    def area(self):
        return 3.14159 * self._radius ** 2

circle = Circle(5)
print(circle.area)  # 78.53975
circle.radius = 10  # Uses setter

### Static method and class method

In [ ]:
class MathUtils:
    class_variable = "I'm a class variable"
    
    @staticmethod
    def add(a, b):
        # No access to self or cls
        return a + b
    
    @classmethod
    def get_class_info(cls):
        # Access to cls (the class itself)
        return f"This is {cls.__name__}: {cls.class_variable}"

print(MathUtils.add(2, 3))  # 5
print(MathUtils.get_class_info())  # This is MathUtils: I'm a class variable

### lru_cache from functools

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def expensive_function(n):
    # Simulating expensive computation
    print(f"Computing for {n}")
    return n ** 2

print(expensive_function(5))  # Computing for 5, returns 25
print(expensive_function(5))  # Returns 25 (from cache, no computation)